In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from evallm.utils.bootstrap import boostrap_mean

from evallm.experiments.evaluate_tokenization_regexp_non_icl import *
import re
import pandas as pd

from evallm.experiments.models_display import model_by_display_key

In [3]:
# from permacache import sync_all_caches

# sync_all_caches()

In [4]:
def for_models(*model_keys, amount=100):
    return {
        model_key: evaluate_model_regexp_matching_multiple(
            model_by_display_key[model_key], amount
        )
        for model_key in model_keys
    }

In [5]:
all_model_results = {
    **for_models(
        "gpt-4o",
        "gpt-4o-mini",
        "claude-3.5",
        "o3-mini",
        "gpt-3.5-chat",
        "gpt-3.5-instruct",
        "qwen-2.5-coder-7B",
        "mistral-nemo-minitron-8B",
        # amount=,
    ),
    # **for_models("mistral-nemo-minitron-8B", amount=10),
}

In [6]:
summary(*all_model_results["gpt-4o"])

Percent positive: 53%
Percent correct : 100%


In [7]:
all_model_accs = {k: (p == t).mean() for k, (p, t) in all_model_results.items()}

In [8]:
all_model_accs_keys = list(all_model_accs)
all_model_accs_values = [all_model_accs[k] for k in all_model_accs_keys]
pd.DataFrame(
    all_model_accs_values, index=all_model_accs_keys, columns=["acc"]
).sort_values("acc")

,acc
gpt-3.5-instruct,0.67
mistral-nemo-minitron-8B,0.69
gpt-3.5-chat,0.82
qwen-2.5-coder-7B,0.91
o3-mini,1.00
claude-3.5,1.00
gpt-4o-mini,1.00
gpt-4o,1.00


In [9]:
sorted(set(model_by_display_key) - set(all_model_results))

['codestral-22B',
 'deepseek-coder-33b-instruct',
 'falcon-7b',
 'gemma-7b',
 'llama3-70B',
 'llama3-8B',
 'llama3.1-70B',
 'llama3.1-8B',
 'llama3.1-8B-Instruct',
 'mistral-nemo-base-12B',
 'mistral-nemo-instruct-12B',
 'qwen-2.5-32B',
 'qwen-2.5-7B',
 'qwen-2.5-coder-instruct-32B',
 'qwen-2.5-coder-instruct-7B',
 'starcoder2-15b']